In [165]:
import numpy as np
from random import shuffle

#################################################################################
######################################SETUP######################################
#################################################################################
numbers=(1,1,1,2,2,3,3,4,4,5)
colors=('r','y','g','b','w')           #possibly add multi... "m"
ManyCol=len(colors)

#CREATE A MATRIX OF FIVE HANDS, ONE ON EACH ROW WITH CARDS REPRESENTED BY SERIAL NUMBERS FROM
#THE ORIGINAL DECK, TOP TO BOTTOM, STARTING AT 0. WE DEAL EACH HAND FROM THE TOP OF THE DECK.
table=np.arange(20).reshape(5,4)
#[A,B,C,D,E]=table  #MAY USE LETTERING OF PLAYERS, BUT PRLY NOT

#CREATE A FULL SHUFFLED DECK WITH NOMENCLATURE AS IN [2,0] FOR "3 red." THINK OF THE INDICES IN deck
#AS SERIAL NUMBERS WRITTEN ON THE BACKS OF THE CARDS. DeckOut CONVERTS TO EXTERNAL FORMAT like "r3".
def CreateDeck():
    deck=[]
    for color in range(ManyCol):
        for number in numbers:
            deck.append([color,number-1])    #internal format numbers range 0-4; colors are numbered
    shuffle(deck)
    #deck=tuple(deck)        #converts deck from list to tuple (mutable to immutable)
    return deck             #...maybe keep as list if want multiple deals

deck=CreateDeck()
DeckOut=[colors[card[0]]+str(card[1]+1) for card in deck]
for i in range(int(len(deck)/4)+1):          #PRINTS DECK 4 AT A TIME
    print(DeckOut[4*i:4*(i+1)])
####################################################################################################
#####################################PUBLIC DATA####################################################
####################################################################################################

#############           stacks/extras******************for logging played/publicly visible cards 
#THE ARRAY stacks GIVES THE LAST NUMBER SUCCESSFULLY PLAYED FOR EACH COLOR AND extras IS A
#ManyColx5 ARRAY TELLING THE NUMBER OF EXTRAS OF EACH CARD. FOR EXAMPLE, A 5 COLOR GAME WITH ONLY
#A 1R DISCARD AND 1Y PLAYED MIGHT LOOK LIKE stacks=[0,1,0,0,0] AND
#extras=[[1,1,1,1,0],[1,1,1,1,0],[2,1,1,1,0],[2,1,1,1,0],[2,1,1,1,0]]. 
stacks=[[1,0,0,0,0]]*ManyCol
extras=[[2,1,1,1,0]]*ManyCol

#THE ROWS IN THE ARRAY priority TELL THE CARDS SERIAL NUMBERS IN THE ORDER THAT PLAYERS
#WILL SEARCH. TOP PRIORITY ARE CARDS CLUED BOTH NUMBER AND COLOR. NEXT COME NUMBER OR COLOR. THEN,
#UNCLUED CARDS. CHRONOLOGICAL ORDER BREAKS TIES. FOR EXAMPLE, IF A PLAYER HAS BEEN TOLD CARDS 6
#AND 8 ARE 4's, BUT DOESN'T KNOW ABOUT THE OTHER CARDS 21, 22, THEN HER SEARCH PRIORITY WILL BE
#priority[2]=[8,6,22,21]. 6 AND 8 HAVE BEEN CLUED, SO THEY GET PRIORITY OVER 21 AND 22. THEN, THE NEWER
#CARDS 8 AND 22 GET PRIORITY OVER 6 AND 21 (RESP.) THEN PLAYER 2 WILL SEARCH FOR PLAYS FROM 8 TO 21. 
#PRIORITY CAN CHANGE IN CERTAIN SITUATIONS, SUCH AS A CLUED CARD MOVING TO THE LOWEST PRIORITY WHEN IT IS
#KNOWN TO BE DEAD. THE LOWEST PRIORITY CARD IS CALLED CHOP BECAUSE IT IS UP NEXT FOR DISCARD.
priority=table[:,::-1]

#############           PubInfo/NumClues*********************for logging public clue information
#THE COORDINATES (i,j,k,l) IN THE ARRAY PubInfo TELL WHETHER PLAYER i THINKS IT'S
#POSSIBLE, BASED ONLY ON PUBLIC INFORMATION (STACKS, CLUES, AND DISCARDS) FOR THEIR SERIAL j
#TO BE COLOR k AND NUMBER l. THE LIST NumClues TELLS THE NUMBER OF CLUES ON EACH CARD,
#ORDERED THE SAME AS deck(BY SERIAL NUMBER). 0:NO CLUE; 1:NUMBER XOR COLOR;2:NUMBER AND COLOR.
#THIS AIDS IN DETERMINING priority WHEN INGENSTING A NEW CLUE OR DRAWING A NEW CARD.
PubInfo=np.ones(shape=(5,len(deck),ManyCol,5), dtype='uint8', order='C')
NumClues=[0]*len(deck)

 
####################################################################################################
#####################################PRIVATE DATA###################################################
####################################################################################################

#THE COORDINATES (i,j,k) IN THE ARRAY AllClues TELLS WHETHER PLAYER i CAN SEE THAT A COLOR j/NUMBER k
#CARD WAS EVER CLUED. WHEN A CLUED CARD IS DISCARDED, AllClues FORGETS IT WAS EVER CLUED.
AllClues=np.zeros(shape=(5,ManyCol,5),dtype='uint8',order='C')

#THE ROWS IN THE PRIVATE MATRIX status DESCRIBE EACH PERSPECTIVE ABOUT CARDS.
#NUMBERS AND STATUSES ARE TENTATIVE.
# -900) DISCARDED TO TRASH OR UNSUCCESSFULLY PLAYED
# -800) SURVIVOR, UNSAVED
# -700) ENDANGERED TWINLESS OR DOUBLE CHOP, UNSAVED
# -125) DEAD (UNKNOWN TO PLAYER)
# -100) DEAD (KNOWN TO PLAYER)
#  000) DEFAULT (DRAW PILE OR UNNOTEWORTHY)
#  025) MARKED (NUMBER AND/OR COLOR)
#  050) TWINLESS 4 (NOT ENDANGERED)
#  075) TWINLESS 3 (NOT ENDANGERED)
#  100) EXPECTED TO PLAY IN QUEUE
#  125) EXPECTED TO PLAY NEXT (NOT 250)
#  250) EXPECTED TO IGNITE CHAIN (UNMARKED)
#  900) PLAYED TO STACK
##################################status=np.zeros((5,len(deck)))



#THE PRIVATE ARRAY info HOLDS THE PERSPECTIVES OF EACH OF THE FIVE PLAYERS.
#COORDINATES (i,j,k) POINT TO AN 8 BIT STRING CORRESPONDING TO INFORMATION
#THAT PLAYER i KNOWS ABOUT PLAYER j'S HAND WITH RESPECT TO CLUE k. THE FIRST
#4 BITS HAVE NEGATIVE INFORMATION FOR THE CORRESPONDING 4 CARDS IN PLAYER j'S
#HAND. THE LAST 4 BITS HAVE POSITIVIE INFORMATION.
###################################info=np.ndarray(shape=(5,5,10), dtype=np.uint8, order='C')         #For 6 colors, need shape=5,11

#THE PRIVATE ARRAY metainfo HOLDS METAINFORMATION ABOUT EACH HAND, SIMILAR TO info.
#COORDINATES (i,j,k,l) DESCRIBE IN THE SAME WAY AS info WHAT PLAYER i THINKS
#PLAYER j KNOWS ABOUT PLAYER k'S HAND WITH RESPECT TO CLUE l.
#####################################metainfo=np.ndarray(shape=(5,5,5,10), dtype=np.uint8, order='C')

['g1', 'w4', 'r4', 'b3']
['g1', 'y3', 'r3', 'y1']
['b1', 'r2', 'b5', 'w2']
['w5', 'w4', 'g2', 'r2']
['y1', 'w1', 'g4', 'y5']
['w2', 'w1', 'w3', 'b2']
['r4', 'g2', 'y4', 'b2']
['r1', 'r3', 'b3', 'g3']
['w1', 'r5', 'g5', 'b1']
['b4', 'r1', 'y1', 'y2']
['g1', 'b1', 'y3', 'r1']
['g3', 'b4', 'y2', 'y4']
['g4', 'w3']


In [166]:
#TEST 5 COLOR CASE
def TEST():
    global deck, stacks, status, priority
    deck=[[0,4],[3,2],[1,1],[1,0],[0,1],[2,0],[2,3],[0,2],[4,0],[4,0],[3,3],[1,2],[1,4],[0,3],[0,2],[4,1],[0,0],[2,2],[1,3],[2,0]]+[[0,0]]*10
    status=[[1,0,0,0,0]]*ManyCol      #status edit?
    
    DeckOut=[colors[card[0]]+str(card[1]+1) for card in deck]
    print('DeckOut:')
    for i in range(5):
        print(DeckOut[4*i:4*(i+1)])

In [167]:
def play(color):
    stacks[color]=[0]+stacks[color][:-1]

def Mod5SkipBack(value, skip, SkipAlso=-1):
    while True:
        value=(value-1)%5
        if value not in {skip,SkipAlso}:
            return value

def RecordClue(player, value, ColOrNum):
    gather=[]
    if ColOrNum:
        print('Recording Number %u to player %u'%(value, player))
    else:
        print('Recording Color %c to player %u'%(colors[value], player))
    for card in range(4):
        if deck[table[player,card]][ColOrNum]==value:
            gather.append(1)
        else: gather.append(0)
    print('RecordClue gathered ', gather,'\n')
    if ColOrNum:
        upd=[np.subtract([1,1,1,1],gather)]*5
        upd[value]=gather
        upd=np.transpose([upd]*ManyCol,axes=(2,0,1))
        PubInfo[player,table[player],:,:]=np.logical_and(PubInfo[player,table[player],:,:],upd)
    else:
        upd=[np.subtract([1,1,1,1],gather)]*ManyCol
        upd[value]=gather
        upd=np.transpose([upd]*5,axes=(2,1,0))
        PubInfo[player,table[player],:,:]=np.logical_and(PubInfo[player,table[player],:,:],upd)
    print('update matrix for player', player,':\n', upd)

def react(player, color, number, queue=[]):
    print('looking for color %c and number %u in player %u'%(colors[color], number, player))
    print('and skipping serials', queue,'... priority is ', priority[player]) 
    for suspect in priority[player]:
        if suspect in queue:
            continue
        elif PubInfo[player,suspect,color,number]:
            print('returning', suspect)
            return suspect

###Start at clue giver going backward to look for a possible chain.. once a card is found, look at that same player
#for the next card and keep going backward until the clued card is reached
def respond(player, giver, color, number, IsSelf=0):
    print('player %u responds to color %c and number %u from player %u'%(player, colors[color], number, giver))
    i=stacks[color].index(1)
    queue=[]
    other=Mod5SkipBack(giver, player)
    print('the next card is numbered %u and the first player to search is %u'%(i, other))
    fail=0                          #####Would like to allow giver to make chains involving cards in his hand
    while i<number:
        WouldPlay=react(other, color, i, queue)
        print('Player %u would play %u in response'%(player, WouldPlay))
        if deck[WouldPlay]==[color,i]:
            print('It matches color %c and number %u'%(colors[color], i))
            queue.append(WouldPlay)
            i+=1
            fail=0
            print('new number and fail count are', i, fail)
        else:
            print('It did not match color %c and number %u'%(colors[color], i))
            other=Mod5SkipBack(other,player,giver)
            fail+=1
            print('new fail count is', fail)
        if fail==3:
            print('Did not find needed card in any visible hands. Appending to player', player)
            queue.append(react(player, color, i, queue))
            i+=1
            fail=0
            print('next number and fail count:',i,fail)
    print('returning queue', queue)
    return queue

In [168]:

TEST()

play(0)
play(0)

play(3)
play(3)
play(3)

RecordClue(2,3,1)
RecordClue(2,4,0)   #make RecordClue update priority

respond(0,4,4,1)

DeckOut:
['r5', 'b3', 'y2', 'y1']
['r2', 'g1', 'g4', 'r3']
['w1', 'w1', 'b4', 'y3']
['y5', 'r4', 'r3', 'w2']
['r1', 'g3', 'y4', 'g1']
Recording Number 3 to player 2
RecordClue gathered  [0, 0, 1, 0] 

update matrix for player 2 :
 [[[1 1 1 0 1]
  [1 1 1 0 1]
  [1 1 1 0 1]
  [1 1 1 0 1]
  [1 1 1 0 1]]

 [[1 1 1 0 1]
  [1 1 1 0 1]
  [1 1 1 0 1]
  [1 1 1 0 1]
  [1 1 1 0 1]]

 [[0 0 0 1 0]
  [0 0 0 1 0]
  [0 0 0 1 0]
  [0 0 0 1 0]
  [0 0 0 1 0]]

 [[1 1 1 0 1]
  [1 1 1 0 1]
  [1 1 1 0 1]
  [1 1 1 0 1]
  [1 1 1 0 1]]]
Recording Color w to player 2
RecordClue gathered  [1, 1, 0, 0] 

update matrix for player 2 :
 [[[0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]
  [1 1 1 1 1]]

 [[0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]
  [0 0 0 0 0]
  [1 1 1 1 1]]

 [[1 1 1 1 1]
  [1 1 1 1 1]
  [1 1 1 1 1]
  [1 1 1 1 1]
  [0 0 0 0 0]]

 [[1 1 1 1 1]
  [1 1 1 1 1]
  [1 1 1 1 1]
  [1 1 1 1 1]
  [0 0 0 0 0]]]
player 0 responds to color w and number 1 from player 4
the next card is numbered 0 and the first

[9]